In [7]:
# !pip install requests
# !pip install cv2
# !pip install mediapipe
# !pip install pygame
# !pip install setuptools wheel
# !pip install playsound

In [8]:
#import

import cv2
import mediapipe as mp
import time

from playsound import playsound
import os
import pygame
import numpy as np
from requests.auth import HTTPBasicAuth
from urllib.parse import quote
from requests.auth import HTTPBasicAuth
import math
import requests


In [10]:

def check_thumbs_rx(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_x = all_x[2:5]
    other_x = all_x[5:] #but not 0

    indici_nocche=[5, 9, 13, 17]
    nocche=[all_y[i] for i in indici_nocche]

    indici_tips=[8, 12, 16, 20]
    tips=[all_y[i] for i in indici_tips]


    dita_chiuse = all(
        all_y[tip] > all_y[nocca]
        for tip, nocca in zip(indici_tips, indici_nocche)
    )


    if min(thumb_x) > max(other_x) and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[17].y) < 0.2 and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[9].y) < 0.2 and\
        hand_landmarks.landmark[4].x>hand_landmarks.landmark[3].x and\
        dita_chiuse and\
        min(tips)>max(nocche):
        return True
    else:
        return False
    
    
def check_thumbs_sx(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_x = all_x[2:5]
    other_x = all_x[5:] #but not 0

    indici_nocche=[5, 9, 13, 17]
    nocche=[all_y[i] for i in indici_nocche]
    indici_tips=[8, 12, 16, 20]
    tips=[all_y[i] for i in indici_tips]
    
    dita_chiuse = all(
        all_y[tip] > all_y[nocca]
        for tip, nocca in zip(indici_tips, indici_nocche)
    )

    if min(thumb_x) < max(other_x) and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[17].y) < 0.2 and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[9].y) < 0.2 and\
            hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x and\
            dita_chiuse:
        return True
    else:
        return False

def result_callback(result, output_image, timestamp_ms):
    if result.gestures:
        gesture = result.gestures[0][0].category_name
    elif result.result.hand_world_landmarks:
        print(check_thumbs_rx(result.hand_world_landmarks[0]))
        print(check_thumbs_sx(result.hand_world_landmarks[0]))

    cap = cv2.VideoCapture(0)

    BaseOptions = mp.tasks.BaseOptions
    GestureRecognizer = mp.tasks.vision.GestureRecognizer
    GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
    VisionRunningMode = mp.tasks.vision.RunningMode

    # Create a gesture recognizer instance with the image mode
    options = GestureRecognizerOptions(
        base_options=BaseOptions(model_asset_path='./assets/gesture_recognizer.task'),
        running_mode=VisionRunningMode.LIVE_STREAM,
        result_callback=result_callback,
        num_hands=1)

    recognizer = GestureRecognizer.create_from_options(options)
    prev_timestamp_ms = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        current_timestamp_ms = int(time.time() * 1000)
        if current_timestamp_ms > prev_timestamp_ms:
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
            recognizer.recognize_async(mp_image, current_timestamp_ms)
        
        cv2.imshow("Drivetune", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()